In [1]:
# Reading NIWE GHI measurement data
filename = "/home/l1nx/NIWE/data/Data_20190110.csv"
# Use pandas to do so
import pandas as pd
import numpy as np

# Define timezone
import pytz
tzi = pytz.timezone("Asia/Colombo")

# Read CSV line by line an read the last column
with open(filename) as f:
    lis=[line.split(',') for line in f]        # create a list of lists
ts = [l[0] for l in lis[1:]]

In [2]:
from datetime import datetime
# Convert strings with timestamps to python internal datetime objects
dts = [ datetime.strptime(d, "%d-%m-%Y %H:%M:%S") for d in ts ]
# Convert local datetime to UTC datetime to make it compatible with image timestamps
ldt = [ tzi.localize(dt,is_dst=False).astimezone(pytz.UTC) for dt in dts ]

In [3]:
# Measured GHI
ghi = [ int(l[-2]) for l in lis[1:]]
# Unix Timestamp
time = [ d.timestamp() for d in ldt ]
# Datetime objects
dates = np.array([ pytz.UTC.localize(datetime.utcfromtimestamp(ts)) for ts in time ])

In [4]:
# Interpolate measurements to 1s resolution
ghi = pd.Series(ghi, index = dates)
res = ghi.resample('1S').interpolate()
dates = [ts.to_pydatetime() for ts in res.index]
time = [ dt.timestamp() for dt in dates ]
ghi = np.array(res.values)

In [5]:
# Import clearsky data with pvlib
from pvlib.location import Location
lat = 12.956785    # camera latitude
lon = 80.2142      # camera longitude
loc = Location(lat, lon)
clear = loc.get_clearsky(res.index, model="simplified_solis")
clear2 = loc.get_clearsky(res.index, model="ineichen")
clear3 = loc.get_clearsky(res.index, model="haurwitz")



In [10]:
# Visualize data with matplotlib
import matplotlib.pyplot as plt
%matplotlib auto
plt.plot(dates, ghi)
plt.plot(dates,clear['ghi'].values,label="Simplified Solis")
plt.plot(dates,clear2['ghi'].values,label="Ineichen")
plt.plot(dates,clear3['ghi'].values,label="Haurwitz")
plt.legend()
plt.show()

Using matplotlib backend: TkAgg


In [8]:
import tables
import os
import pvlib
import calendar
pvlib_path = os.path.dirname(os.path.abspath(pvlib.clearsky.__file__))
filepath = os.path.join(pvlib_path, 'data', 'LinkeTurbidities.h5')
def plot_turbidity_map(month, vmin=1, vmax=100):
   plt.figure();
   with tables.open_file(filepath) as lt_h5_file:
       ltdata = lt_h5_file.root.LinkeTurbidity[:, :, month-1]
   plt.imshow(ltdata, vmin=vmin, vmax=vmax);
   # data is in units of 20 x turbidity
   plt.title('Linke turbidity x 20, ' + calendar.month_name[month]);
   plt.colorbar(shrink=0.5);
   plt.tight_layout();


In [9]:
plot_turbidity_map(1)
